In [1]:
import os
from azure.core.credentials import AzureKeyCredential
from azure.ai.documentintelligence import DocumentIntelligenceClient

from dotenv import load_dotenv

load_dotenv()

endpoint = os.getenv("AZURE_DOCUMENT_INTELLIGENCE_ENDPOINT")
api_key = os.getenv("AZURE_DOCUMENT_INTELLIGENCE_API_KEY")

client = DocumentIntelligenceClient(endpoint, AzureKeyCredential(api_key))

In [2]:
file_path = "data/Azure OpenAI Service モデル - Azure OpenAI _ Microsoft Learn.pdf"

with open(file_path, "rb") as f:
    poller = client.begin_analyze_document(
        "prebuilt-layout",
        body=f,
        content_type="application/octet-stream",
        output_content_format="MARKDOWN",
    )
result = poller.result()

## マークダウン出力の結果

In [3]:
result_markdown = result.content
base_name = os.path.splitext(os.path.basename(file_path))[0]
_file_path = f"./data/{base_name}.md"

with open(_file_path, "w", encoding="utf-8") as f:
    f.write(result_markdown)

## テーブル

In [ ]:
tables = result.tables
#display(tables)

[{'rowCount': 10, 'columnCount': 2, 'cells': [{'kind': 'columnHeader', 'rowIndex': 0, 'columnIndex': 0, 'content': 'モデル', 'boundingRegions': [{'pageNumber': 1, 'polygon': [0.725, 2.1306, 2.115, 2.1306, 2.115, 2.3975, 0.725, 2.3975]}], 'spans': [{'offset': 259, 'length': 3}], 'elements': ['/paragraphs/4']}, {'kind': 'columnHeader', 'rowIndex': 0, 'columnIndex': 1, 'content': '説明', 'boundingRegions': [{'pageNumber': 1, 'polygon': [2.115, 2.1306, 7.5276, 2.1306, 7.5276, 2.3975, 2.115, 2.3975]}], 'spans': [{'offset': 272, 'length': 2}], 'elements': ['/paragraphs/5']}, {'rowIndex': 1, 'columnIndex': 0, 'content': '○ シリーズ モデル', 'boundingRegions': [{'pageNumber': 1, 'polygon': [0.725, 2.3975, 2.115, 2.3975, 2.115, 2.6573, 0.725, 2.6503]}], 'spans': [{'offset': 295, 'length': 10}], 'elements': ['/paragraphs/6']}, {'rowIndex': 1, 'columnIndex': 1, 'content': '高度な問題解決、増強された集中力と能力を備えた推論モデル。', 'boundingRegions': [{'pageNumber': 1, 'polygon': [2.115, 2.3975, 7.5276, 2.3975, 7.5276, 2.6503, 2.115, 2

In [5]:
import pandas as pd

table_list = []

for table in tables:
    rows = {}
    for item in table["cells"]:
        row_index = item.get("rowIndex")
        col_index = item.get("columnIndex")
        content = item.get("content")
        rows.setdefault(row_index, {})[col_index] = content
    df = pd.DataFrame.from_dict(rows, orient="index")
    df.columns = df.iloc[0]  # 1行目をカラム名に設定
    df = df.drop(0).reset_index(drop=True)
    display(df)
    table_list.append(df)

,モデル,説明
0,○ シリーズ モデル,高度な問題解決、増強された集中力と能力を備えた推論モデル。
1,GPT-4o、GPT-4o mini、GPT- 4 Turbo,最新の最も能力の高い Azure OpenAl モデルであり、テキストと画像の両方を入力とし...
2,GPT-4o audio,"低遅延、""音声入力、音声出力"" の会話のやり取り、またはオーディオ生成をサポートする GPT..."
3,GPT-4,GPT-3.5 を基に改善され、自然言語とコードを理解し、生成できるモデルのセット。
4,GPT-3.5,GPT-3 を基に改善され、自然言語とコードを理解し、生成できるモデルのセット。
5,埋め込み,テキストを数値ベクトル形式に変換して、テキストの類似性を促進できるモデルのセット。
6,DALL-E,自然言語からオリジナルの画像を生成できるモデルのシリーズ。
7,Whisper,音声を文字起こしして音声テキスト変換を翻訳できる一連のモデル。
8,テキスト読み上げ(プレビュ ) ー,テキストを音声に合成できるプレビュー段階の一連のモデル。


,モデル ID,説明,最大要求(トー クン),トレーニング デ ータ(最大)
0,o3-mini (2025- 01-31),最新の推論モデルであり、推論能力が強化されています。 - 構造化出力 - テキストのみの処理...,"入力:200,000 出力:100,000",2023年10月
1,o1 (2024-12-17),01 シリーズの中で最も能力の高いモデルで、推論能力が強化されています。 - 構造化出力 -...,"入力:200,000 出力:100,000",2023年10月
2,o1-preview (2024-09-12),以前のプレビュー バージョン,"入力:128,000 出力:32,768",2023年10月
3,o1-mini (2024- 09-12),01 シリーズの中のより速く、よりコスト効率の高いオプションであり、速度を必要としリソー ス...,"入力:128,000 出力:65,536",2023年10月


,モデル,リージョン
0,o3-mini,「モデル テーブル」を参照してください。
1,o1,「モデル テーブル」を参照してください。
2,o1- preview,「モデル テーブル」を参照してください。このモデルを使用できるのは、元の制限付きアクセスの一...
3,o1-mini,「モデル テーブル」を参照してください。


,モデル ID,説明,最大要求(トーク ン),トレーニング データ(最 大)
0,gpt-4o-mini-audio-preview (2024-12-17) GPT-4o ...,オーディオとテキスト生成向けのオーディオ モデル。,"入力:128,000 出力:4,096",2023年10月
1,gpt-4o-mini-realtime-preview (2024-12- 17) GPT...,リアルタイム オーディオ処理向けのオーディオ モデ ル。,"入力:128,000 出力:4,096",2023年10月
2,gpt-4o-audio-preview (2024-12-17) GPT-4o audio,オーディオとテキスト生成向けのオーディオ モデル。,"入力:128,000 出力:4,096",2023年10月
3,gpt-4o-realtime-preview (2024-12-17) GPT-4o audio,リアルタイム オーディオ処理向けのオーディオ モデ ル。,"入力:128,000 出力:4,096",2023年10月
4,gpt-4o-realtime-preview (2024-10-01) GPT-4o audio,リアルタイム オーディオ処理向けのオーディオ モデ ル。,"入力:128,000 出力:4,096",2023年10月


,モデル,リージョン
0,gpt-4o-mini-audio-preview,米国東部 2 (グローバル標準)
1,gpt-4o-mini-realtime-preview,米国東部 2 (グローバル標準) スウェーデン中部(グローバル標準)
2,gpt-4o-audio-preview,米国東部 2 (グローバル標準) スウェーデン中部(グローバル標準)
3,gpt-4o-realtime-preview,米国東部 2(グローバル標準) スウェーデン中部(グローバル標準)


...

,リージョン,o1- preview.,o1- mini- 2024- 09-12,gpt- 40% 2024 年5,gpt- 40.,"gpt- 4o- mini, 2024-",gpt- 4. 0613,gpt-4. 1106- Preview,gpt-4. 0125- Preview,gpt-4. vision- preview,gpt- 4. turbo- 2024-,gpt- 4- 32k. 0613,"gpt-35- turbo, 0301",gpt-35- turbo. 0613,gpt-35- turbo. 1106,gpt-35 turbo. 0125
0,,,NaN,月 13 ⽇,,07-18,,,NaN,,04-09,,NaN,NaN,NaN,
1,,,NaN,NaN,,,NaN,,NaN,NaN,,NaN,NaN,NaN,,NaN
2,australiaeast,-,-,-,-,-,✅ :selected:,✅ :selected:,-,✅ :selected:,-,✅ :selected:,-,✅ :selected:,✅ :selected:,✅ :selected:
3,canadaeast,-,-,-,-,-,✅ :selected:,✅ :selected:,-,-,-,✅ :selected:,-,✅ :selected:,✅ :selected:,✅ :selected:
4,eastus,✅ :selected:,✅ :selected:,✅ :selected:,✅ :selected:,✅ :selected:,✅ :selected:,-,✅ :selected:,-,✅ :selected:,-,✅ :selected:,✅ :selected:,-,✅ :selected:
5,eastus2,✅ :selected:,✅ :selected:,✅ :selected:,✅ :selected:,✅ :selected:,✅ :selected:,✅ :selected:,-,-,✅ :selected:,-,-,✅ :selected:,-,✅ :selected:
6,francecentral,-,-,-,-,-,✅ :selected:,✅ :selected:,-,-,-,✅ :selected:,✅ :selected:,✅ :selected:,✅ :selected:,-
7,japaneast,-,-,-,-,-,-,-,-,✅ :selected:,-,-,-,✅ :selected:,-,✅ :selected:
8,northcentralus,✅ :selected:,✅ :selected:,✅ :selected:,✅ :selected:,✅ :selected:,✅ :selected:,-,✅ :selected:,-,✅ :selected:,-,-,✅ :selected:,-,✅ :selected:
9,norwayeast,-,-,-,-,-,-,✅ :selected:,-,-,-,-,-,-,-,-


,モデル,リージョン
0,gpt-4 (0314) gpt-4-32k (0314),米国東部
1,gpt-4 (0613) gpt-4-32k (0613),米国東部
2,NaN,フランス中部
3,NaN,米国中南部
4,NaN,英国南部
5,NaN,米国東部 2
6,NaN,東日本
7,NaN,英国南部


,モデル ID,微調整リージョン,最大要求(トークン),トレーニング データ(最大)
0,gpt-35-turbo (0613),米国東部 2,"4,096",2021 年9月
1,gpt-35-turbo (1106),米国東部 2,"入力:16,385",2021 年9月
2,gpt-35-turbo (0125),米国東部 2 米国中北部,"16,385",2021 年9月
3,gpt-4 (0613) 1,米国中北部 スウェーデン中部,8192,2021 年9月
4,gpt-4o-mini (2024-07-18),米国中北部 スウェーデン中部,"入力:128,000 出力:16,384 トレーニング例のコンテキスト長: 64,536",2023年10月
5,gpt-4o (2024-08-06),米国東部 2,"入力:128,000",2023年10月
6,NaN,米国中北部,NaN,NaN
7,NaN,スウェーデン中部,,NaN
8,NaN,スイス西部,NaN,NaN
9,NaN,米国中北部,"出力:4,096",NaN


,リージョン,gpt- 4o、 2024 年 5 月 13 ⽇,gpt- 4o、 2024- 08-06,gpt-4o- mini- 2024-07- 18,gpt- 4. 0613,gpt-4. 1106- Preview,gpt-4. 0125- Preview,gpt-4. turbo- 2024-04- 09,gpt-4- 32k. 0613,gpt-35- turbo- 0613,gpt-35- turbo- 1106,gpt-35- turbo- 0125,"gpt-35- turbo- 16k, 0613"
0,australiaeast,-,-,-,:selected: ✅,:selected: ✅,-,-,:selected: ✅,:selected: ✅,:selected: ✅,✅ :selected:,:selected: ✅
1,eastus,:selected: ✅,:selected: ✅,:selected: ✅,-,-,:selected: ✅,:selected: ✅,-,:selected: ✅,-,:selected: ✅,:selected: ✅
2,eastus2,:selected: ✅,:selected: ✅,:selected: ✅,-,:selected: ✅,-,:selected: ✅,-,:selected: ✅,-,:selected: ✅,:selected: ✅


,リージョン,gpt- 40. 2024 年 5 月 13 ⽇,gpt- 40% 2024- 08-06,gpt-4o- mini- 2024-07- 18,gpt- 4. 0613,gpt-4. 1106- Preview,gpt-4. 0125- Preview,gpt-4. turbo- 2024-04- 09,"gpt-4- 32k, 0613",gpt-35- turbo- 0613,"gpt-35- turbo, 1106","gpt-35- turbo, 0125","gpt-35- turbo- 16k, 0613"
0,francecentral,-,-,-,✅ :selected:,✅ :selected:,-,-,✅ :selected:,✅ :selected:,✅ :selected:,-,✅ :selected:
1,japaneast,-,-,-,-,-,-,-,-,✅ :selected:,-,✅ :selected:,✅ :selected:
2,norwayeast,-,-,-,-,✅ :selected:,-,-,-,-,-,-,-
3,southindia,-,-,-,-,✅ :selected:,-,-,-,-,✅ :selected:,✅ :selected:,-
4,swedencentral,✅ :selected:,✅ :selected:,✅ :selected:,✅ :selected:,✅ :selected:,-,✅ :selected:,✅ :selected:,✅ :selected:,✅ :selected:,-,✅ :selected:
5,uksouth,-,-,-,-,✅ :selected:,✅ :selected:,-,-,✅ :selected:,✅ :selected:,✅ :selected:,✅ :selected:
6,westus,✅ :selected:,✅ :selected:,✅ :selected:,-,✅ :selected:,-,✅ :selected:,-,-,✅ :selected:,✅ :selected:,-
7,westus3,✅ :selected:,✅ :selected:,✅ :selected:,-,✅ :selected:,-,✅ :selected:,-,-,-,✅ :selected:,-


## セクション

In [6]:
sections = result.sections
display(sections)

[{'spans': [{'offset': 0, 'length': 31939}], 'elements': ['/sections/1', '/sections/5', '/sections/6', '/sections/7', '/sections/8', '/sections/9', '/sections/10', '/sections/12', '/sections/13', '/sections/15', '/sections/17', '/sections/18', '/sections/19', '/sections/20', '/sections/21', '/sections/22', '/sections/23', '/sections/24', '/sections/28']},
 {'spans': [{'offset': 0, 'length': 2763}], 'elements': ['/paragraphs/0', '/sections/2', '/sections/4']},
 {'spans': [{'offset': 28, 'length': 2365}], 'elements': ['/paragraphs/1', '/paragraphs/2', '/paragraphs/3', '/tables/0', '/sections/3']},
 {'spans': [{'offset': 1055, 'length': 1338}], 'elements': ['/paragraphs/24', '/paragraphs/25', '/paragraphs/26', '/tables/1', '/paragraphs/47', '/paragraphs/48', '/paragraphs/49', '/paragraphs/50', '/paragraphs/51']},
 {'spans': [{'offset': 2396, 'length': 367}], 'elements': ['/paragraphs/52', '/paragraphs/53', '/tables/2']},
 {'spans': [{'offset': 2766, 'length': 2168}], 'elements': ['/paragr

## パラグラフ

In [ ]:
paragraphs = result.paragraphs
#display(paragraphs)

[{'spans': [{'offset': 0, 'length': 25}], 'boundingRegions': [{'pageNumber': 1, 'polygon': [0.7108, 0.5478, 4.3491, 0.5286, 4.3507, 0.8329, 0.7124, 0.8522]}], 'role': 'sectionHeading', 'content': 'Azure OpenAI Serviceモデル'},
 {'spans': [{'offset': 28, 'length': 22}], 'boundingRegions': [{'pageNumber': 1, 'polygon': [0.7132, 0.8964, 1.9597, 0.8923, 1.9601, 1.0161, 0.7136, 1.0202]}], 'role': 'sectionHeading', 'content': '[アーティクル]·2025/02/28'},
 {'spans': [{'offset': 52, 'length': 173}], 'boundingRegions': [{'pageNumber': 1, 'polygon': [0.7165, 1.1728, 7.4862, 1.1673, 7.4867, 1.6646, 0.7169, 1.6702]}], 'content': 'Azure OpenAl Service では、さまざまな機能と価格ポイントを備えた多様なモデルセットが利用されています。モデルの可用性はリージ ョンとクラウドごとに異なります。Azure Government モデルの可用性については、Azure Government の OpenAl Service に関するセク ションを参照してください。'},
 {'spans': [{'offset': 227, 'length': 12}], 'boundingRegions': [{'pageNumber': 1, 'polygon': [6.3952, 1.8929, 7.4546, 1.8953, 7.4543, 2.0213, 6.3949, 2.0189]}], 'content': '〔〕 テーブルを展開する'},
 {'spans': [{'of

## セクション分割
- セクション情報に格納されているパラグラフとテーブルを結合する
- セクションの中にセクションが含まれている場合、再帰的にセクションを格納せず、無視する(そうすることで重複なくセクション分割が可能)

In [8]:
# セクションの参照文字列と sections の対応（0 番目はルート用）
section_map = {}
# 1～の sections 要素を "/sections/1", "/sections/2", ... に対応させる
for i in range(1, len(sections)):
    key = f"/sections/{i}"
    section_map[key] = dict(sections[i])


# 再帰的にツリー構造を構築する関数
def build_tree(node, ref=None):
    """
    node が文字列の場合は参照（"/sections/...", "/paragraphs/...", "/tables/..."）とみなし、
    対応する内容を取得する。node が dict（セクション）なら、子要素も再帰的に処理する。
    ref: セクションの参照文字列（例: "/sections/2"）を保持するための引数
    """
    if isinstance(node, str):
        if node.startswith("/sections/"):
            sec = section_map.get(node)
            if sec:
                return build_tree(sec, ref=node)
            else:
                return {"type": "section", "ref": node, "error": "Not found"}
        elif node.startswith("/paragraphs/"):
            pid = int(node.split("/")[-1])
            # paragraphs 辞書から取得
            content = paragraphs[pid].content
            return {"type": "paragraph", "ref": node, "content": content}
        elif node.startswith("/tables/"):
            tid = int(node.split("/")[-1])
            content = table_list[tid].to_markdown()
            return {"type": "table", "ref": node, "content": content}
        else:
            return {"type": "unknown", "ref": node}
    elif isinstance(node, dict) and "elements" in node:
        # セクションの dict の場合、子要素それぞれに対して再帰的に処理
        children = [
            build_tree(child_ref, ref=child_ref) for child_ref in node["elements"]
        ]
        result = {"type": "section", "spans": node.get("spans"), "children": children}
        if ref is not None:
            result["ref"] = ref
        return result
    else:
        return node


# ツリーの表示関数（インデント付きで再帰的に表示）
def print_tree(node, prefix=""):
    node_type = node.get("type", "unknown")
    if node_type == "section":
        spans = node.get("spans", [{}])
        offset = spans[0].get("offset", "N/A")
        length = spans[0].get("length", "N/A")
        # ここでは ref が無い場合は [section] と表示
        ref = node.get("ref", "[section]")
        print(f"{prefix}{ref} (offset={offset}, length={length})")
        children = node.get("children", [])
        for i, child in enumerate(children):
            branch = "├─ " if i < len(children) - 1 else "└─ "
            print_tree(child, prefix + "    " + branch)
    elif node_type in ("paragraph", "table"):
        print(f"{prefix}{node.get('ref')} : {node.get('content')}")
    else:
        print(f"{prefix}{node}")


# ルートは sections[0]（ルートコンテナ）を使う
root_node = dict(sections[0])
tree = build_tree(root_node)
print_tree(tree)



[section] (offset=0, length=31939)
    ├─ /sections/1 (offset=0, length=2763)
    ├─     ├─ /paragraphs/0 : Azure OpenAI Serviceモデル
    ├─     ├─ /sections/2 (offset=28, length=2365)
    ├─     ├─     ├─ /paragraphs/1 : [アーティクル]·2025/02/28
    ├─     ├─     ├─ /paragraphs/2 : Azure OpenAl Service では、さまざまな機能と価格ポイントを備えた多様なモデルセットが利用されています。モデルの可用性はリージ ョンとクラウドごとに異なります。Azure Government モデルの可用性については、Azure Government の OpenAl Service に関するセク ションを参照してください。
    ├─     ├─     ├─ /paragraphs/3 : 〔〕 テーブルを展開する
    ├─     ├─     ├─ /tables/0 : |    | モデル                            | 説明                                                                                                                                             |
|---:|:----------------------------------|:-------------------------------------------------------------------------------------------------------------------------------------------------|
|  0 | ○ シリーズ モデル                 | 高度な問題解決、増強された集中力と能力を備えた推論モデル。                          

In [9]:
def extract_leaf_sections(node, accumulated=""):
    """
    ツリー内のleafセクション（子にセクションがないセクションノード）を再帰的に探し、
    親ノードの直接のコンテンツを子ノードの内容の先頭にマージして、最終的なセクションIDと
    content をまとめた dict のリストを返す。

    出力例:
      {
         "section": 2,
         "content": "親のコンテンツ\n子のコンテンツ"
      }
    """
    results = []
    if node.get("type") == "section":
        # 現在のセクションの直接のコンテンツ（段落やテーブル）の抽出
        direct_contents = []
        for child in node.get("children", []):
            if child.get("type") in ("paragraph", "table"):
                direct_contents.append(child.get("content", ""))
        direct_text = "\n".join(direct_contents).strip()
        # 祖先から受け継いだ内容に現在の直接コンテンツを追加
        new_accumulated = (
            (accumulated + "\n" + direct_text).strip() if direct_text else accumulated
        )

        # 子にセクションが存在するかチェック
        child_sections = [
            child
            for child in node.get("children", [])
            if child.get("type") == "section"
        ]
        if not child_sections:
            # これはleafセクション
            ref = node.get("ref", "")
            section_id = None
            if ref.startswith("/sections/"):
                try:
                    section_id = int(ref.split("/")[-1])
                except Exception:
                    section_id = ref
            results.append({"section": section_id, "content": new_accumulated})
        else:
            # 子セクションがある場合、累積した内容を渡して再帰的に探索
            for child in child_sections:
                results.extend(extract_leaf_sections(child, new_accumulated))
    return results


# 例: 既に構築済みの tree データからleafセクションを抽出する
leaf_sections = extract_leaf_sections(tree)

# 結果表示
for sec in leaf_sections:
    print("###############"+str(sec['section'])+"###############")
    print(sec['content'])

###############3###############
Azure OpenAI Serviceモデル
[アーティクル]·2025/02/28
Azure OpenAl Service では、さまざまな機能と価格ポイントを備えた多様なモデルセットが利用されています。モデルの可用性はリージ ョンとクラウドごとに異なります。Azure Government モデルの可用性については、Azure Government の OpenAl Service に関するセク ションを参照してください。
〔〕 テーブルを展開する
|    | モデル                            | 説明                                                                                                                                             |
|---:|:----------------------------------|:-------------------------------------------------------------------------------------------------------------------------------------------------|
|  0 | ○ シリーズ モデル                 | 高度な問題解決、増強された集中力と能力を備えた推論モデル。                                                                                       |
|  1 | GPT-4o、GPT-4o mini、GPT- 4 Turbo | 最新の最も能力の高い Azure OpenAl モデルであり、テキストと画像の両方を入力として受け入れることができるマルチ モーダル バージョンを備えています。 |
|  2 | GPT-4o audio                      | 低遅延、"音声入力、音声出力" の会話のやり取り、またはオーディオ生成をサポートす

In [10]:
leaf_sections

[{'section': 3,
  'content': 'Azure OpenAI Serviceモデル\n[アーティクル]·2025/02/28\nAzure OpenAl Service では、さまざまな機能と価格ポイントを備えた多様なモデルセットが利用されています。モデルの可用性はリージ ョンとクラウドごとに異なります。Azure Government モデルの可用性については、Azure Government の OpenAl Service に関するセク ションを参照してください。\n〔〕 テーブルを展開する\n|    | モデル                            | 説明                                                                                                                                             |\n|---:|:----------------------------------|:-------------------------------------------------------------------------------------------------------------------------------------------------|\n|  0 | ○ シリーズ モデル                 | 高度な問題解決、増強された集中力と能力を備えた推論モデル。                                                                                       |\n|  1 | GPT-4o、GPT-4o mini、GPT- 4 Turbo | 最新の最も能力の高い Azure OpenAl モデルであり、テキストと画像の両方を入力として受け入れることができるマルチ モーダル バージョンを備えています。 |\n|  2 | GPT-4o audio                      | 低遅延、"音声入力、音声出力" の会話のやり取り、またはオーディオ生成

In [12]:
import json

_file_path = f"./data/{base_name}.json"
with open(_file_path, 'w', encoding='utf-8') as f:
    json.dump(leaf_sections, f, ensure_ascii=False, indent=4)
